In [24]:
# Documentation: https://research.donorschoose.org/t/download-opendata/33
import ml
reload(ml)
from ml import *
import timeit
import scipy
import operator
import numpy as np
import pandas as pd
import cPickle as pickle
from scipy import stats
import seaborn as sns
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
from pprint import pprint
from __future__ import division
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder  
from sklearn.utils import resample
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
plt.style.use('classic')
%matplotlib inline

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

'''
    Analysis originaly performed in Python 2 (deprecated)
    Seaborn, Statsmodel, and * imports broken in Python 3
'''

## Methods

In [2]:
def cov(arr):
    return np.std(arr) / np.mean(arr) 

def fundingTime(arr):
    return (arr.max() - arr.min()).days

def interEventTime(arr):
    return [(t - s).seconds for s, t in zip(sorted(arr), sorted(arr)[1:])]

def avgInterEventTime(arr):
    return np.mean(interEventTime(arr))

def covInterEventTime(arr):
    varr = interEventTime(arr)
    return np.std(varr) / np.mean(varr) 

# Initialize Label Encoder
le = LabelEncoder()
# Initialize Standard Scaler
scaler = StandardScaler() 
# projectsgeq4 = np.loadtxt('projectsgeq4contributions.out', delimiter=',', dtype=str)
projectsall = np.loadtxt('projectsall.out', delimiter=',', dtype=str)

## Donations

In [3]:
donations = pd.read_csv('../Data/donations.gz', escapechar='\\', names=['_donationid', '_projectid', '_donor_acctid', '_cartid', 'donor_city', 'donor_state', 'donor_zip', 'is_teacher_acct', 'donation_timestamp', 'donation_to_project', 'donation_optional_support', 'donation_total', 'donation_included_optional_support', 'payment_method', 'payment_included_acct_credit', 'payment_included_campaign_gift_card', 'payment_included_web_purchased_gift_card', 'payment_was_promo_matched', 'is_teacher_referred', 'giving_page_id', 'giving_page_type', 'for_honoree', 'thank_you_packet_mailed'])
# Filter relevant columns
columns = ["_donationid", "_projectid", "_donor_acctid", "donation_timestamp", "donation_total", "payment_method"]
donations = donations[columns]
# Convert donation timestamp to datetime format
donations["donation_timestamp"] = pd.to_datetime(donations.donation_timestamp)
# Variable /YYYY/MM to allow monthly time unit analysis
donations["time_trunc"] = donations.donation_timestamp.map(lambda x: x.strftime('%Y%m')) 
# List of project_ids for projects that received re-allocated funds
filtered = list(set(donations[donations.payment_method == 'no_cash_received']["_projectid"]))
# Exclude these donations for donation level analysis
donations = donations[donations.payment_method != 'no_cash_received'] 
# Drop temporary column
donations = donations.drop(['payment_method'], axis=1)
# Show sample data
donations.sample(5)

,_donationid,_projectid,_donor_acctid,donation_timestamp,donation_total,time_trunc
994053,28ede5c8c8fe6b6af0d8bb0313c30c17,f8b2848191166c090f727d109f01dfdd,bb4aed2151296282a644a65e541da29d,2007-11-27 12:23:47.516,50.000,200711
2261309,5d2c3487caf28e1fc26f9d4536912566,e429f379b38178028333bb4315d039e0,1665f065802108cc0fd1fdddc75f578a,2010-12-28 22:29:33.690,25.000,201012
1097812,2d36fe6c633864b29b3dad575ac068e5,2c080db7d96400f01560de0de8c589ad,922f9b701bfe745011eba7d8975c6377,2013-11-19 20:40:17.656,25.000,201311
2547170,68f42b0edbff09f6cea632b8868cdd96,bcc93fa53007527575d6f2ef209a99c9,7fbbd53cb78bb7a35525d6918c6b5704,2015-09-13 09:46:35.756,2.000,201509
2096056,565e2b7b2ea60e5333429f2b0c2f8561,671c5c02ef8176cbdfcded038180990b,a4c5ccb8a63a455e93df7b61e3eac656,2009-06-22 14:20:47.575,141.000,200906


In [4]:
print "Contributors:", donations._donor_acctid.nunique()

Contributors: 1523962


In [5]:
print len(projectsall)

215825


### Filter Donations

In [6]:
def filter_donations_by_amount(threshold):
    donations_ = []
    threshold = threshold
    for project_id in tqdm_notebook(projectsall):
        var_df = donations.query("_projectid == '{0}'".format(project_id)).sort_values(by='donation_timestamp')
        series = list(var_df['donation_total'].fillna(0))
        norm_cdf = np.cumsum(series)
        try:
            limit = max(series)*threshold
            if norm_cdf[0] > limit:
                pass
            else:
                 for i in range(len(norm_cdf)):
                    if limit < norm_cdf[i]:
                        donations_.append(var_df[:i-1])  
                        break
        except Exception as e:
            pass
        
    return pd.concat(donations_, axis=0)

In [7]:
def filter_donations(threshold):  
    donations_ = []
    threshold = threshold
    for project_id in tqdm_notebook(projectsgeq4):
        var_df = donations.query("_projectid == '{0}'".format(project_id)).sort_values(by='donation_timestamp')
        donations_.append(var_df[:int(len(var_df)*threshold)])
    return pd.concat(donations_, axis=0)

In [8]:
# donations_25 = filter_donations_by_amount(0.25)
# donations_25.to_csv("../Data/donations_25.csv", index=False)
# donations_50 = filter_donations_by_amount(0.5)
# donations_50.to_csv("../Data/donations_50.csv", index=False)
# donations_75 = filter_donations_by_amount(0.75)
# donations_75.to_csv("../Data/donations_75.csv", index=False)
# donations_100 = filter_donations_by_amount(1)
# donations_100.to_csv("../Data/donations_100.csv", index=False)

In [9]:
# donations_25 = pd.read_csv("../Data/donations_25.csv")
# donations_50 = pd.read_csv("../Data/donations_50.csv")
# donations_75 = pd.read_csv("../Data/donations_75.csv")
# donations_100 = pd.read_csv("../Data/donations_100.csv")

In [10]:
len(donations_25), len(donations_50), len(donations_75), len(donations_100)

(56918, 125482, 186646, 263821)

### Donations Aggregation

In [11]:
f = {'_projectid': ['max'],
     '_donationid': ['count'], # Contributions Count
     '_donor_acctid':['count'], # Contributor Count
     'donation_timestamp': ['min', avgInterEventTime, covInterEventTime], # First Contribution Timestamp, AvgInterEventTime, CovInterEventTime
     'donation_total': ['min', 'mean', 'max', cov] # Minimum, Mean, Maximum, CoV
}

#### Donations 25%

In [12]:
donations_25 = donations_25.drop(['time_trunc'], axis=1)
donations_25 = donations_25[~donations_25['_projectid'].isin(filtered)] # Exclude these projects for project level analysis
donations_25 = pd.DataFrame(donations_25.groupby('_projectid').agg(f).as_matrix())
donations_25 = donations_25.rename(index=str, columns={0: "MinimumContributionAmount", 
                                               1: "MeanContributionAmount",
                                               2: "MaxContributionAmount",
                                               3: "CovContributionAmount",
                                               4: "FirstContributionTimestamp",
                                               5: "AvgInterEventTime_sec",
                                               6: "CovInterEventTime",
                                               7: "ProjectID",
                                               8: "NumContributions",
                                               9: "NumContributors"})
donations_25 = donations_25[donations_25.MinimumContributionAmount >=0]
donations_25.head()

,MinimumContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,ProjectID,NumContributions,NumContributors
0,75.000,75.000,75.000,0.000,2016-05-21 21:25:00.685000,NaN,NaN,00002eb25d60a09c318efbd0797bffb5,1,1
1,10.000,20.000,25.000,0.354,2015-03-22 08:38:41.548000,5556.000,1.000,00048c5627b7a599d81a424bcdc8c2eb,3,3
2,25.000,25.000,25.000,0.000,2016-01-10 14:56:07.828000,NaN,NaN,0008e601cd7d067abbe30c7f4ec589cd,1,1
3,20.000,20.000,20.000,0.000,2015-10-17 12:45:06.590000,NaN,NaN,000b4e95067c0bf756a213ad7698d3a3,1,1
4,10.000,10.000,10.000,0.000,2014-09-17 10:19:13.655000,NaN,NaN,000d06fe5336e85574a0e5d78269c4e5,1,1


#### Donations 50%

In [13]:
donations_50 = donations_50.drop(['time_trunc'], axis=1)
donations_50 = donations_50[~donations_50['_projectid'].isin(filtered)] # Exclude these projects for project level analysis
donations_50 = pd.DataFrame(donations_50.groupby('_projectid').agg(f).as_matrix())
donations_50 = donations_50.rename(index=str, columns={0: "MinimumContributionAmount", 
                                               1: "MeanContributionAmount",
                                               2: "MaxContributionAmount",
                                               3: "CovContributionAmount",
                                               4: "FirstContributionTimestamp",
                                               5: "AvgInterEventTime_sec",
                                               6: "CovInterEventTime",
                                               7: "ProjectID",
                                               8: "NumContributions",
                                               9: "NumContributors"})
donations_50 = donations_50[donations_50.MinimumContributionAmount >=0]
donations_50.head()

,MinimumContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,ProjectID,NumContributions,NumContributors
0,75.000,75.000,75.000,0.000,2016-05-21 21:25:00.685000,NaN,NaN,00002eb25d60a09c318efbd0797bffb5,1,1
1,25.000,25.000,25.000,0.000,2015-08-21 18:09:29.092000,NaN,NaN,000226bf2362b3ea92229b29c2edf737,1,1
2,25.000,25.000,25.000,0.000,2010-04-22 08:38:08.065000,NaN,NaN,000238b44c187c254752736e53ee46f2,1,1
3,46.000,46.000,46.000,0.000,2012-02-17 15:34:27.202000,NaN,NaN,00029c282384a5d15bf2184b0578630e,1,1
4,20.000,20.000,20.000,0.000,2015-08-08 16:08:11.163000,NaN,NaN,00046dc723c15f748b48b0d408f167f7,1,1


#### Donations 75%

In [14]:
donations_75 = donations_75.drop(['time_trunc'], axis=1)
donations_75 = donations_75[~donations_75['_projectid'].isin(filtered)] # Exclude these projects for project level analysis
donations_75 = pd.DataFrame(donations_75.groupby('_projectid').agg(f).as_matrix())
donations_75 = donations_75.rename(index=str, columns={0: "MinimumContributionAmount", 
                                               1: "MeanContributionAmount",
                                               2: "MaxContributionAmount",
                                               3: "CovContributionAmount",
                                               4: "FirstContributionTimestamp",
                                               5: "AvgInterEventTime_sec",
                                               6: "CovInterEventTime",
                                               7: "ProjectID",
                                               8: "NumContributions",
                                               9: "NumContributors"})
donations_75 = donations_75[donations_75.MinimumContributionAmount >=0]
donations_75.head()

,MinimumContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,ProjectID,NumContributions,NumContributors
0,75.000,75.000,75.000,0.000,2016-05-21 21:25:00.685000,NaN,NaN,00002eb25d60a09c318efbd0797bffb5,1,1
1,100.000,100.000,100.000,0.000,2012-05-11 00:03:44.415000,NaN,NaN,00005f52c98eeaf92b2414a352b023a4,1,1
2,25.000,25.000,25.000,0.000,2015-08-21 18:09:29.092000,NaN,NaN,000226bf2362b3ea92229b29c2edf737,1,1
3,25.000,25.000,25.000,0.000,2010-04-22 08:38:08.065000,NaN,NaN,000238b44c187c254752736e53ee46f2,1,1
4,45.910,47.303,50.000,0.040,2012-02-17 15:34:27.202000,856.500,0.999,00029c282384a5d15bf2184b0578630e,3,3


#### Donations 100%

In [15]:
donations_100 = donations_100.drop(['time_trunc'], axis=1)
donations_100 = donations_100[~donations_100['_projectid'].isin(filtered)] # Exclude these projects for project level analysis
donations_100 = pd.DataFrame(donations_100.groupby('_projectid').agg(f).as_matrix())
donations_100 = donations_100.rename(index=str, columns={0: "MinimumContributionAmount", 
                                               1: "MeanContributionAmount",
                                               2: "MaxContributionAmount",
                                               3: "CovContributionAmount",
                                               4: "FirstContributionTimestamp",
                                               5: "AvgInterEventTime_sec",
                                               6: "CovInterEventTime",
                                               7: "ProjectID",
                                               8: "NumContributions",
                                               9: "NumContributors"})
donations_100 = donations_100[donations_100.MinimumContributionAmount >=0]
donations_100.head()

,MinimumContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,ProjectID,NumContributions,NumContributors
0,75.000,75.000,75.000,0.000,2016-05-21 21:25:00.685000,NaN,NaN,00002eb25d60a09c318efbd0797bffb5,1,1
1,20.000,60.000,100.000,0.667,2012-05-11 00:03:44.415000,2671.000,0.000,00005f52c98eeaf92b2414a352b023a4,2,2
2,25.000,25.000,25.000,0.000,2016-07-18 20:01:43.691000,NaN,NaN,0000c0bdc0f15bd239cfffa884791a10,1,1
3,25.000,25.000,25.000,0.000,2015-08-21 18:09:29.092000,NaN,NaN,000226bf2362b3ea92229b29c2edf737,1,1
4,25.000,25.000,25.000,0.000,2010-04-22 08:38:08.065000,NaN,NaN,000238b44c187c254752736e53ee46f2,1,1


## Projects

In [16]:
columns = ["_projectid", "total_donations", "funding_status", "date_posted", "date_completed",
           "school_metro", "primary_focus_area", "poverty_level", "grade_level", "resource_type", "teacher_prefix"]
statusmap = {'completed':1, 'expired':0, 'live':2, 'reallocated':3}
metromap = {'rural':1, 'suburban':2, 'urban':3}
projects = pd.read_csv('../Data/projects.gz', escapechar='\\', names=['_projectid', '_teacher_acctid', '_schoolid', 'school_ncesid', 'school_latitude', 'school_longitude', 'school_city', 'school_state', 'school_zip', 'school_metro', 'school_district', 'school_county', 'school_charter', 'school_magnet', 'school_year_round', 'school_nlns', 'school_kipp', 'school_charter_ready_promise', 'teacher_prefix', 'teacher_teach_for_america', 'teacher_ny_teaching_fellow', 'primary_focus_subject', 'primary_focus_area' ,'secondary_focus_subject', 'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'vendor_shipping_charges', 'sales_tax', 'payment_processing_charges', 'fulfillment_labor_materials', 'total_price_excluding_optional_support', 'total_price_including_optional_support', 'students_reached', 'total_donations', 'num_donors', 'eligible_double_your_impact_match', 'eligible_almost_home_match', 'funding_status', 'date_posted', 'date_completed', 'date_thank_you_packet_mailed', 'date_expiration'])
projects = projects[columns]
projects["date_posted"] = pd.to_datetime(projects.date_posted)
projects["date_completed"] = pd.to_datetime(projects.date_completed)
projects = projects.applymap(lambda s: statusmap.get(s) if s in statusmap else s)
projects = projects.applymap(lambda s: metromap.get(s) if s in metromap else s)
projects['school_metro'] = projects['school_metro'].fillna(0)
projects['primary_focus_area'] = le.fit_transform(projects.primary_focus_area)
projects['poverty_level'] = le.fit_transform(projects.poverty_level)
projects['grade_level'] = le.fit_transform(projects.grade_level)
projects['resource_type'] = le.fit_transform(projects.resource_type)
projects['teacher_prefix'] = le.fit_transform(projects.teacher_prefix)
projects = projects.rename(index=str, columns={"_projectid": "ProjectID",
                                               "total_donations": "AmountRaised",
                                               "funding_status": "Status"})
projects.head(5)

,ProjectID,AmountRaised,Status,date_posted,date_completed,school_metro,primary_focus_area,poverty_level,grade_level,resource_type,teacher_prefix
0,7342bd01a2a7725ce033a179d22e382d,251.900,1,2002-09-13,2002-09-23,3.000,46,1,64,54,47
1,ed87d61cef7fda668ae70be7e0c6cebf,137.000,1,2002-09-13,2002-09-23,0.000,49,1,65,52,50
2,b56b502d25666e29550d107bf7e17910,125.000,1,2002-09-16,2002-09-19,0.000,46,3,63,54,50
3,016f03312995d5c89d6b348be4682166,205.000,1,2002-09-17,2002-09-17,3.000,48,0,66,54,50
4,cf6275558534ca1b276b0d8d5130dd9a,264.000,1,2002-09-17,2002-09-23,3.000,46,1,66,54,50


## Merge Data

In [17]:
data_25 = donations_25.merge(projects, on="ProjectID", how="left")
data_25["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_25.FirstContributionTimestamp) - pd.to_datetime(data_25.date_posted)).dt.seconds
data_25["Duration (days)"] = (pd.to_datetime(data_25.date_completed) - pd.to_datetime(data_25.date_posted)).dt.days
data_25 = data_25[data_25["Duration (days)"] >= 0]
data_25 = data_25[data_25.Status != 2]
data_25["Status"] = (data_25.Status == 1).astype("int")
data_25 = data_25.replace([np.inf, -np.inf], np.nan)

data_50 = donations_50.merge(projects, on="ProjectID", how="left")
data_50["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_50.FirstContributionTimestamp) - pd.to_datetime(data_50.date_posted)).dt.seconds
data_50["Duration (days)"] = (pd.to_datetime(data_50.date_completed) - pd.to_datetime(data_50.date_posted)).dt.days
data_50 = data_50[data_50["Duration (days)"] >= 0]
data_50 = data_50[data_50.Status != 2]
data_50["Status"] = (data_50.Status == 1).astype("int")
data_50 = data_50.replace([np.inf, -np.inf], np.nan)

data_75 = donations_75.merge(projects, on="ProjectID", how="left")
data_75["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_75.FirstContributionTimestamp) - pd.to_datetime(data_75.date_posted)).dt.seconds
data_75["Duration (days)"] = (pd.to_datetime(data_75.date_completed) - pd.to_datetime(data_75.date_posted)).dt.days
data_75 = data_75[data_75["Duration (days)"] >= 0]
data_75 = data_75[data_75.Status != 2]
data_75["Status"] = (data_75.Status == 1).astype("int")
data_75 = data_75.replace([np.inf, -np.inf], np.nan)

data_100 = donations_100.merge(projects, on="ProjectID", how="left")
data_100["TimeToFirstContribution (sec)"] = (pd.to_datetime(data_100.FirstContributionTimestamp) - pd.to_datetime(data_100.date_posted)).dt.seconds
data_100["Duration (days)"] = (pd.to_datetime(data_100.date_completed) - pd.to_datetime(data_100.date_posted)).dt.days
data_100 = data_100[data_100["Duration (days)"] >= 0]
data_100 = data_100[data_100.Status != 2]
data_100["Status"] = (data_100.Status == 1).astype("int")
data_100 = data_100.replace([np.inf, -np.inf], np.nan)
# data.head()
#TODO: Drop timestamp columns

## Model Evaluation

In [18]:
features_5 = ["NumContributors", "CovInterEventTime", "CovContributionAmount", "TimeToFirstContribution (sec)", 'Duration (days)']

### Bids 25%

In [19]:
X = data_25[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_25['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 32299, 0: 99})
Completed QDA in 0.1209026 seconds
Completed CART in 0.2960282 seconds
Completed GNB in 0.0857005 seconds
Completed RF in 42.8608174 seconds
Completed LR in 0.3760011 seconds
Completed ADB in 4.3865418 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.803,0.999,0.803,0.890,0.725
1,CART,0.993,0.997,0.996,0.997,0.498
2,GNB,0.835,0.999,0.836,0.910,0.721
3,RF,0.997,0.997,1.000,0.998,0.500
4,LR,0.997,0.997,1.000,0.998,0.500
5,ADB,0.997,0.997,1.000,0.998,0.500


In [28]:
classifier = classifiers.get("RF")
estimates = []
n_size = []
for x in tqdm_notebook(range(0,100)):
    df_majority = data_25[data_25.Status==1]
    df_minority = data_25[data_25.Status==0]
    samples = len(df_minority)
    df_majority_downsampled = resample(df_majority,
                                      replace = False,
                                      n_samples = samples)
    downsampled = pd.concat([df_majority_downsampled, df_minority])
    X = downsampled[features_5].fillna(-1).as_matrix()
    X = scaler.fit_transform(X)
    y = downsampled['Status'].as_matrix()
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
    n_size.append(samples*2)
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "Mean N", np.mean(n_size)

Mean AUC 0.6597474747474749
St. Dev 0.03610072606197576
Mean N 198.0


### Bids 50%

In [20]:
X = data_50[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_50['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 58199, 0: 160})
Completed QDA in 0.2274489 seconds
Completed CART in 0.9420091 seconds
Completed GNB in 0.2560228 seconds
Completed RF in 88.4730204 seconds
Completed LR in 0.6860423 seconds
Completed ADB in 7.0275405 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.849,0.999,0.850,0.918,0.709
1,CART,0.994,0.997,0.997,0.997,0.498
2,GNB,0.865,0.999,0.866,0.928,0.708
3,RF,0.997,0.997,1.000,0.999,0.500
4,LR,0.997,0.997,1.000,0.999,0.500
5,ADB,0.997,0.997,1.000,0.999,0.500


In [29]:
classifier = classifiers.get("RF")
estimates = []
n_size = []
for x in tqdm_notebook(range(0,100)):
    df_majority = data_50[data_50.Status==1]
    df_minority = data_50[data_50.Status==0]
    samples = len(df_minority)
    df_majority_downsampled = resample(df_majority,
                                      replace = False,
                                      n_samples = samples)
    downsampled = pd.concat([df_majority_downsampled, df_minority])
    X = downsampled[features_5].fillna(-1).as_matrix()
    X = scaler.fit_transform(X)
    y = downsampled['Status'].as_matrix()
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
    n_size.append(samples*2)
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "Mean N", np.mean(n_size)

Mean AUC 0.671875
St. Dev 0.02857965841293419
Mean N 320.0


### Bids 75%

In [21]:
X = data_75[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_75['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 72353, 0: 206})
Completed QDA in 0.2104235 seconds
Completed CART in 0.6894502 seconds
Completed GNB in 0.2051059 seconds
Completed RF in 116.8026365 seconds
Completed LR in 0.8165364 seconds
Completed ADB in 8.6251068 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.836,0.998,0.837,0.911,0.688
1,CART,0.994,0.997,0.997,0.997,0.501
2,GNB,0.860,0.998,0.861,0.924,0.695
3,RF,0.997,0.997,1.000,0.999,0.500
4,LR,0.997,0.997,1.000,0.998,0.500
5,ADB,0.997,0.997,1.000,0.999,0.500


In [30]:
classifier = classifiers.get("RF")
estimates = []
n_size = []
for x in tqdm_notebook(range(0,100)):
    df_majority = data_75[data_75.Status==1]
    df_minority = data_75[data_75.Status==0]
    samples = len(df_minority)
    df_majority_downsampled = resample(df_majority,
                                      replace = False,
                                      n_samples = samples)
    downsampled = pd.concat([df_majority_downsampled, df_minority])
    X = downsampled[features_5].fillna(-1).as_matrix()
    X = scaler.fit_transform(X)
    y = downsampled['Status'].as_matrix()
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
    n_size.append(samples*2)
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "Mean N", np.mean(n_size)

Mean AUC 0.6591990291262135
St. Dev 0.022854477711712482
Mean N 412.0


### Bids 100%

In [22]:
X = data_100[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = data_100['Status'].as_matrix()
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({1: 90038, 0: 271})
Completed QDA in 0.2706302 seconds
Completed CART in 1.0876853 seconds
Completed GNB in 0.2391815 seconds
Completed RF in 167.2447071 seconds
Completed LR in 1.3648916 seconds
Completed ADB in 12.6613823 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.847,0.998,0.848,0.917,0.712
1,CART,0.994,0.997,0.997,0.997,0.500
2,GNB,0.861,0.998,0.862,0.925,0.702
3,RF,0.997,0.997,1.000,0.998,0.500
4,LR,0.997,0.997,1.000,0.998,0.500
5,ADB,0.997,0.997,1.000,0.998,0.500


In [31]:
classifier = classifiers.get("RF")
estimates = []
n_size = []
for x in tqdm_notebook(range(0,100)):
    df_majority = data_100[data_100.Status==1]
    df_minority = data_100[data_100.Status==0]
    samples = len(df_minority)
    df_majority_downsampled = resample(df_majority,
                                      replace = False,
                                      n_samples = samples)
    downsampled = pd.concat([df_majority_downsampled, df_minority])
    X = downsampled[features_5].fillna(-1).as_matrix()
    X = scaler.fit_transform(X)
    y = downsampled['Status'].as_matrix()
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
    n_size.append(samples*2)
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "Mean N", np.mean(n_size)

Mean AUC 0.69
St. Dev 0.01973217689529376
Mean N 542.0
